# Detection and Segmentation

Semantic Segmentation, Classification+Localization, Object Detection, Instance Segmentation에 대한 개략적인 내용은 Mask R-CNN Project를 진행하면서 이미 다룬 바 있다. 아래 주소를 통해 그 내용을 다시 확인할 수 있다. 이번 Report는 위의 내용에 대해서 조금 더 구체적인 내용과 기존에 미처 알지 못했던 부분을 짚어보고자 한다.

https://github.com/YoonSungLee/Mask-R-CNN-Project_AI-Innovation-Square/blob/master/Mask%20R-CNN%20Report.ipynb

# Semantic Segmentation

### Fully Convolutional

Semantic Segmentation은 Sliding Window를 이용한 방법과 Fully Convolutional을 이용한 방법으로 나뉠 수 있다. 우리는 후자의 방법에 주목하고자 한다. 초기 아이디어는 아래 그림과 같이 단순히 모든 픽셀을 한 번에 예측하기 위해 여러 개의 Convolutional Layer를 사용하는 것이었다.

<img src="https://i.imgur.com/QavfiXj.png" width="100%">

하지만 이러한 방법은 오리지널 사이즈의 이미지를 지속적으로 Convolutional Layer에 입력시켜야 하기 때문에 비용이 매우 많이 든다는 단점을 가지고 있다. 결국 깊은 신경망을 구성하기가 어려울 것이다. 이에 대한 해결책으로 Convolutional Layer를 구성하되 네트워크 내부에 downsampling과 upsampling을 추가시켜 low resolution을 처리하도록 한다. 이 때 Downsampling에는 Pooling, Strided Convolution이 있으며, Upsampling에는 Unpooling, Max Unpooling, Strided Transpose Convolution 등이 있다. 이 방법을 통해 계산 효율을 높일 수 있으며, 네트워크를 더 깊게 만드는 것을 가능하게 한다. 해당 그림은 아래와 같다.

<img src="https://i.imgur.com/5FGs9tM.png" width="100%">

# In-Network upsampling

### Unpooling

<img src="https://i.imgur.com/SsCHYEW.png" width="100%">

참고로 Keras의 UpSampling2D는 위의 Nearest Neighbor방법을 사용하고 있다는 것을 경험적으로 확인했다. 그 결과는 다음과 같다.

In [0]:
import tensorflow as tf
import numpy as np

In [2]:
x = np.array([[1,3],[5,7]])
x = x.reshape(1,2,2,1)

print(x.shape)
print(x)

(1, 2, 2, 1)
[[[[1]
   [3]]

  [[5]
   [7]]]]


In [3]:
y = tf.keras.layers.UpSampling2D((2,2))(x)

print(y.shape)
print(y)

(1, 4, 4, 1)
tf.Tensor(
[[[[1]
   [1]
   [3]
   [3]]

  [[1]
   [1]
   [3]
   [3]]

  [[5]
   [5]
   [7]
   [7]]

  [[5]
   [5]
   [7]
   [7]]]], shape=(1, 4, 4, 1), dtype=int64)


### Max Unpooling

<img src="https://i.imgur.com/Kieg11i.png" width="100%">

### Transpose Convolution

위의 세 가지 방법(Nearest Neighbor, Bed of Nails, Max Unpooling)은 '고정된 함수'로써 별도로 학습을 시키지 않는다. 반면에 Transpose Convolution은 학습가능한 Upsampling 방법이라고 할 수 있겠다.

<img src="https://i.imgur.com/BzhFH64.png" width="100%">

위의 그림과 같이 3x3 transpose convolution, stride 2, padding 1의 연산을 수행한다고 해보자. Input feature map의 빨간색 부분의 값을 filter에 곱해 Output 그림의 빨간 박스가 되고, Input feature map의 파란색 부분의 값을 filter에 곱해 Output 그림의 파란 박스가 되는 식으로 진행된다. 즉, Input feature map의 값들은 filter에 곱해지는 가중치 역할을 해준다. 이 때 겹치는 부분은 두 값을 더해준다.<br>
Sum where output overlaps은 충분히 문제가 될 만한 요소이다. 겹치는 부분마다 두 값을 더해주면서 특정 거리를 주기로 값이 커지는 현상이 발생하는데 마치 격자무늬와 같은 모습을 보여 'checkerboard artifacts'라고 한다. 몇몇 논문에서는 4x4 stride 2 혹은 2x2 stride 2를 사용해서 문제를 완화시키기도 한다.<br>
Transpose의 다른 말로 Deconvolution(다른 용어와 혼동되므로 사용을 지양), Upconvolution, Fractionally stride convolution, Backward strided convolution이라고도 한다.

위의 작동원리가 이해가 잘 되지 않는다면 아래 그림의 1D Example을 통해 수월하게 이해할 수 있을것이다.

<img src="https://i.imgur.com/DSduWXM.png" width="100%">

# Classification + Localization

<img src="https://i.imgur.com/nVib0pK.png" width="100%">

이 문제는 기존 Classification 문제에서 Image 한 장 안에 하나의 객체가 있을 것으로 가정하고 Localization 문제가 추가된 형태이다. 학습 이미지는 객체에 대한 카테고리 레이블과 해당 객체의 Bounding Box GT를 동시에 가지고 있는 형태이다. 이 모델은 2개의 Loss가 존재하는데, 따라서 이를 Multitask Loss라고 부른다. Classification에 해당하는 Softmax Loss는 클래스 분류를 얼마나 잘 했는가를 평가하는 지표로써 기존 Classification과 작동방식이 동일하다. 추가적으로 Localization에 해당하는 L2 Loss는 객체의 Bounding Box를 얼마나 잘 찾았는지를 평가하는 지표로써 (x, y, w, h)에 대한 값이 존재하며 회귀문제의 접근방법과 같은 방법을 취한다. 이후 이 두 Loss를 더해서 최종 Loss로 평가하는데, CS231N 수업에서 한 학생이 이에 대한 질문을 하는 장면을 볼 수 있었다. 내용은 아래와 같다.

Q) 두 개의 Loss의 단위가 달라서 Gradient 계산에 문제가 될 소지가 있지 않은가?<br><br>

A) 이 두 개의 Loss를 합친 Loss를 Multi-task Loss라고 한다. 우선 그래디언트를 구하려면 네트워크 가중치들의 각각의 미분 값(scalar)을 계산해야한다. 이제는 Loss가 두 개이니 미분 값(scaler)도 두 개이고 이 두 개를 모두 최소화시켜야 한다. 실제로는 두 Losses의 가중치를 조절하는 하이퍼파라미터가 있다. 두 Losses의 가중치 합이 최종 Loss이다. 그리고 이 두 Losses의 가중 합에 대한 그레디언트를 계산하는 것이다. 하지만 두 Losses간의 가중치를 결정하는 것은 상당히 까다롭다. 이 가중치는 하이퍼파라미터로써 우리가 설정해줘야만 한다. 이 하이퍼파라미터는 지금까지 살펴본 하이퍼파라미터들보다는 상당히 다르다. 이 하이퍼파라미터는 실제로 손실함수의 값 자체를 바꿔버리기 때문이다. 지금까지는 하이퍼파라미터를 다양하게 조절하면서 Loss가 어떻게 변하는지를 관찰했다. 하지만 이 경우에는 하이퍼파라미터의 값이 변하면 Loss 자체의 속성이 변하므로 비교하기가 상당히 까다롭다. 따라서 이 하이퍼파라미터를 조절하는 것은 상당히 어렵다. 따라서 이 하이퍼파라미터를 설정하는 적절한 방법은 문제에 따라 다를 수 있다. 다만 일반적인 방법은 Loss 값으로 비교할 게 아니라 다른 성능지표를 도입하는 것이다. Cross Validation으로 하이퍼파라미터를 최적화할 때 Loss가 아니라 실제 모델의 성능지표를 봐야 할 것이다.

특히 이 모델을 사용할때는 앞부분의 ConvNet을 ImageNet으로 pretrained된 모델을 그대로 사용하고 뒷부분의 FC를 학습시켜 문제에 적합한 모델로 변경시키곤 한다. 이를 우리는 Transfer Learning이라고 부른다.

# Object Detection

<img src="https://i.imgur.com/AQE5RaJ.png" width="100%">

Object Detection 문제는 R-CNN 계열 모델을 다루면서 접했던 문제인데, 이에 대한 배경을 알아두는 것이 흐름을 이해하는데 좋을 것이다. 우리가 이전 문제(Classification + Localization)를 해결할 때 위의 첫 번째 그림을 사용했다. 즉, 이미지 한 장에 객체가 하나 있을것이라고 가정하고 해당 객체의 (x, y, w, h)를 L2로 학습시켰다. 반면에 Object Detection문제는 이미지 한 장에 객체가 얼마나 존재할것인지 알 수 없다는 점에서 더 어려운 문제라고 할 수 있겠다. 만약 두 번째 그림처럼 3개의 객체를 검출하려면 각 객체마다 (x, y, w, h)가 필요할 것이므로 4x4 = 16개의 ouput이 필요하다. 그렇다면 많은 수의 객체라면? 모든 객체의 (x, y, w, h)를 구하는 것은 쉽지 않을 뿐더러, 객체가 얼마나 많은지 모르기 때문에 output의 수를 정해놓을 수 없다.

### Sliding Window

이에 대한 첫 번째 해결책으로 'Sliding Window'가 있다. 픽셀마다 움직이면서 이미지의 크롭을 따서 모두 CNN에 적용하는 방식인데, 이를 통해 각 크롭이 어떤 객체인지, 만약 객체가 아니라면 배경인지를 구분할 수 있겠다. 가능은 하겠지만 이는 굉장히 많은 계산량을 필요로 한다는것을 직감할 수 있을것이다.

### Region Proposal(Selective Search)

이는 단순히 모든 픽셀에 대하여 크롭을 따는 것이 아니라, 객체가 있을 만한 곳의 Region을 찾아내는 방법이다. CPU로 2초간 Selective Search를 돌리면 객체가 있을만한 2000개의 Region Proposal을 만들어낸다. 이 방법은 노이즈가 아주 심하다. 대부분은 실제 객체가 아니겠지만 Recall은 아주 높다. 따라서 이미지에 객체가 존재한다면 Selective Search의 Region Proposal 안에 속할 가능성이 높다. 이후 추출해 낸 Region Proposal을 CNN에 적용하는 방식이다. R-CNN과 Fast R-CNN에서 이 방법을 사용했다.

### Region Proposal Network

이 방법은 기존의 Selective Search가 하던 역할을 Neural Network로 구성하여 모델 안에 들어가있는 구조인데, Faster R-CNN과 Mask R-CNN에서 이 방법을 사용했다. Selective Search와 RPN 관련해서는 마찬가지로 Mask R-CNN Project를 진행하면서 이미 다룬 내용이므로 아래 주소를 통해 자세한 내용을 확인할 수 있다.

https://github.com/YoonSungLee/Mask-R-CNN-Project_AI-Innovation-Square/blob/master/Mask%20R-CNN%20Report.ipynb

### Detection without Proposals: YOLO/SSD

* Faster R-CNN은 RPN으로 먼저 Regression문제를 풀고 ROI 단위로 Classification을 하는 방식(Region-based Method)이다.
* Single Shot Methods는 단 한번에 forward pass만으로 끝내버린다.

Huang et al, “Speed/accuracy trade-offs for modern convolutional object detectors”, CVPR 2017<br><br>

"Faster R-CNN 계열의 Region based methods는 정확도가 높긴 하나 Single Shot Methods보다는 속도가 느리다. Single Shot Methods는 RoI당 별도의 연산을 요구하지 않기 때문이다."

<img src="https://i.imgur.com/sv6GQk8.png" width="100%">

# Reference

CS231n: Convolutional Neural Networks for Visual Recognition